In [1]:
from openpiv import tools, pyprocess, validation, filters, scaling 

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import cv2 # Added for image equalozation
from math import *
import imageio
import os
import pandas as pd
import copy

folder = '/home/matteo/PIV/PIV'
save = 1 # Save images at the end of postProcessing in Images/ImagesCorrected
minus_average = 0 # Consider images cleared of average

In [2]:
# Camera
resolution = (992,1004) #pixel
physical_window = (45,45) # m
pixel_depth = 8 # bit
dynamic_range = 2**pixel_depth # levels
scaling_factor = 1/(np.mean(physical_window)/np.mean(resolution)) # m/pixel

In [3]:
# Load saveVectorField functiopon
%run -i '/home/matteo/PIV/PIV/Scripts/saveVectorField'
%run -i '/home/matteo/PIV/PIV/Scripts/flash_mask'

In [4]:
# Generate strings for the images
numbers = ["{0:03}".format(i) for i in range(2,101)]

In [5]:
# Initialize u and v
x,y,u,v,snr,mask = saveVectorField('030', save=save, minus_average=minus_average, fill = 'checkerboard')
u, v = u.flatten(), v.flatten()

Processing image number:030


/home/matteo/PIV/PIV/Scripts/saveVectorField.py:72: UserWarning: 2 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')
/home/matteo/PIV/PIV/Scripts/saveVectorField.py:75: UserWarning: 2 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')


In [6]:
# Save vector field for all the images

for i in numbers:
    _, _, uv, vv, _, _ = saveVectorField(i, save=save, minus_average=minus_average)
    uv = uv.flatten()
    vv = vv.flatten()
    u = np.c_[u,uv]
    v = np.c_[v,vv]

Processing image number:002
Processing image number:003
Processing image number:004
Processing image number:005


/home/matteo/PIV/PIV/Scripts/saveVectorField.py:72: UserWarning: 1 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')


Processing image number:006


/home/matteo/PIV/PIV/Scripts/saveVectorField.py:75: UserWarning: 0 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')


Processing image number:007
Processing image number:008
Processing image number:009
Processing image number:010
Processing image number:011
Processing image number:012


/home/matteo/PIV/PIV/Scripts/saveVectorField.py:75: UserWarning: 1 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')


Processing image number:013


/home/matteo/PIV/PIV/Scripts/saveVectorField.py:72: UserWarning: 2 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')
/home/matteo/PIV/PIV/Scripts/saveVectorField.py:75: UserWarning: 2 flashed regions in frame a have been replaced with a random noise
  warnings.warn(str(number_a)+ ' flashed regions in frame a have been replaced with a random noise')


Processing image number:014
Processing image number:015
Processing image number:016
Processing image number:017
Processing image number:018
Processing image number:019
Processing image number:020
Processing image number:021
Processing image number:022
Processing image number:023
Processing image number:024
Processing image number:025
Processing image number:026
Processing image number:027
Processing image number:028
Processing image number:029
Processing image number:030
Processing image number:031
Processing image number:032
Processing image number:033


KeyboardInterrupt: 

In [ ]:
# Calculate the mean velocity
umean, vmean = np.mean(u,axis=1), np.mean(v,axis=1)
umean, vmean = umean.reshape(mask.shape), vmean.reshape(mask.shape)

# Initialize mask and snr fields
mask = np.zeros(mask.shape, dtype=bool)
snr = np.zeros(mask.shape)

#save in the simple ASCII table format
tools.save(x, y, umean, vmean, snr, mask, folder + '/Vector_field/exp1_' + 'meanField.txt' )

fig, ax = plt.subplots(figsize=(8,8))
tools.display_vector_field( folder + '/Vector_field/exp1_' + 'meanField.txt' , 
                           ax=ax, scaling_factor=scaling_factor, 
                           scale= 5000, # scale defines here the arrow length
                           width= 0.002, # width is the thickness of the arrow
                           on_img=False, # overlay on the image
                           image_name= 'None');


In [ ]:
# Calculate rms velocity field
#urms, vrms = np.std(u,axis=1), np.std(v,axis=1)
urms, vrms = urms.reshape(mask.shape), vrms.reshape(mask.shape)

#save in the simple ASCII table format
tools.save(x, y, urms, vrms, snr, mask, folder + '/Vector_field/exp1_' + 'rmsField.txt' )

# Plot
X, Y = np.meshgrid(x[0], y.transpose()[0]) 

ax1 = plt.subplot(1,2,1)
ax1.pcolormesh(X,Y,urms)
ax1.axis('equal')
ax2 = plt.subplot(1,2,2)
ax2.axis('equal')
ax2.pcolormesh(X,Y,vrms)
